In [12]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50  import preprocess_input

In [13]:
import warnings

# Ignore FutureWarning from numpy
warnings.simplefilter(action='ignore', category=FutureWarning)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";

# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# Allow growth of GPU memory, otherwise it will always look like all the memory is being used
physical_devices = tf.config.experimental.list_physical_devices('GPU')

tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [15]:
datagenerator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=30,
    vertical_flip=True,
    brightness_range=[0.90,1.45]
    #fill_mode='nearest'
)

In [16]:
# load and iterate training dataset
train_data = datagenerator.flow_from_directory('../train/', 
    class_mode='binary',
    target_size=(250, 250), 
    batch_size=64, 
    shuffle=False)

# load and iterate validation dataset
val_data = datagenerator.flow_from_directory('../valid/', 
    class_mode='binary',
    target_size=(250, 250),
    batch_size=64, 
    shuffle=False)

# load and iterate test dataset
test_data = datagenerator.flow_from_directory('../test/', 
    class_mode='binary',
    target_size=(250, 250),
    batch_size=64, 
    shuffle=False)

Found 2600 images belonging to 2 classes.
Found 870 images belonging to 2 classes.
Found 864 images belonging to 2 classes.


In [17]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.activations import relu, softmax, sigmoid, swish
from tensorflow.keras.optimizers import RMSprop

In [18]:
pre_trained_model = VGG16(input_shape=(250,250,3),
                                include_top=False,
                                weights="imagenet")

# Some weights in later layers are unfreezed
for layer in pre_trained_model.layers[:-5]:
    layer.trainable=False

tf.random.set_seed(2022)

model = tf.keras.models.Sequential([
    pre_trained_model,
    GlobalAveragePooling2D(),    
    Dense(512,activation="swish"),
    Dropout(0.5),
    Dense(256,activation="relu"),
    Dropout(0.5),
    Dense(128, activation='relu'),  
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(learning_rate=1e-4),
              loss="binary_crossentropy",
              metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)              

In [20]:
model.compile(loss='binary_crossentropy', optimizer= RMSprop(learning_rate=0.001), metrics=['acc'])

In [21]:
history = model.fit(
    train_data,
    steps_per_epoch=40,
    epochs=5,
    validation_data=val_data,  
)

2022-11-21 10:26:29.104937: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 48000000 exceeds 10% of free system memory.


Epoch 1/5


2022-11-21 10:26:38.093267: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 48000000 exceeds 10% of free system memory.


 2/40 [>.............................] - ETA: 2:37 - loss: 1.3633 - acc: 0.3789 

2022-11-21 10:26:55.736761: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 48000000 exceeds 10% of free system memory.


...

 5/40 [==>...........................] - ETA: 4:01 - loss: 1.1015 - acc: 0.4652

2022-11-21 10:27:27.368616: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.29GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-11-21 10:27:27.788857: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.29GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


40/40 [==============================] - ETA: 0s - loss: 0.7751 - acc: 0.6724

2022-11-21 10:34:20.172333: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.23GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-11-21 10:34:20.615847: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.29GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


40/40 [==============================] - 482s 12s/step - loss: 0.7728 - acc: 0.6749 - val_loss: 0.6266 - val_acc: 0.7989
Epoch 2/5
40/40 [==============================] - 442s 11s/step - loss: 0.6186 - acc: 0.7956 - val_loss: 0.5794 - val_acc: 0.7989
Epoch 3/5
40/40 [==============================] - 446s 11s/step - loss: 0.5568 - acc: 0.8275 - val_loss: 0.5446 - val_acc: 0.7989
Epoch 4/5
40/40 [==============================] - 443s 11s/step - loss: 0.5622 - acc: 0.7727 - val_loss: 0.5226 - val_acc: 0.7989
Epoch 5/5
40/40 [==============================] - 446s 11s/step - loss: 0.5258 - acc: 0.7909 - val_loss: 0.5085 - val_acc: 0.7989


In [22]:
model.evaluate(test_data)

14/14 [==============================] - 130s 9s/step - loss: 0.5075 - acc: 0.7998


[0.5075328946113586, 0.7997685074806213]